#### Import dependancies and set environment determinism

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

2024-04-12 17:35:37.257995: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


378


In [2]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

#### define SNN

In [3]:
def run_model(model_path,l1,l2,loss):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )

    model_history = model.fit(
        trfeat,
        trlabs,
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model



# # if os.path.isdir(res_dir) == False:

# def run_model(model_path,l1,l2,loss,n):
#     normalizer = preprocessing.Normalization(axis=-1)
#     normalizer.adapt(np.array(trfeat[n]))

#     model = gl.build_dnn_model(
#         normalizer, learning_rate = 0.01, 
#         layer_1 = l1, layer_2 = l2,loss = loss
#     )

#     model_history = model.fit(
#         trfeat[n],
#         trlabs[n],
#         validation_split=0.2,
#         callbacks = [callback],
#         verbose=0, 
#         epochs=500
#     )

#     model_results = model.evaluate(tefeat[n],telabs[n],verbose = 0)
# #     model_filename = os.path.join(model_path)
#     print(model_history)
#     print(model_results)
#     model.save(model_path)
    
#     return model, model_results, model_history

#### Create callback function to quit training if loss does not improve after 10 iterations

In [4]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

#### Define path to save arch test models

In [5]:
home_path = ''

arch_test_path = 'arch_testing'

path = os.path.join(home_path,arch_test_path,)

In [6]:
df = gl.coregister_data('4')
# df = df[:-7]
df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)
df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)

dft = df.copy()

trfeat, tefeat, trlabs, telabs = gl.split_data(df)

In [7]:
l1_list = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
l2_list = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]
loss_list = ['mse','mae']
df = gl.coregister_data('4')
df = df.iloc[:,:-7]

df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)

df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)

dft = df.copy()

trfeat, tefeat, trlabs, telabs = gl.split_data(df)
print(trfeat,tefeat,trlabs,telabs)
for loss in loss_list:
    for l2 in l2_list:
        for l1 in l1_list:
            if l2 >= l1:
                continue
            else:

                model = {}
                model_history = {}
                normalizer = {}
#                 path = '/data/fast1/glacierml/models/LOO_loss_testing/'
                model_path = os.path.join(
                   path ,loss,str(l1)+'-'+str(l2)
                )
                res_dir = os.path.join(path, loss,'final_results.pkl')
                if os.path.isdir(res_dir) == True:
                    continue

                isdir = model_path
                if os.path.isdir(isdir) == True:
                    continue
                if os.path.isdir(isdir) == False:
                    print(model_path)
                    set_global_determinism(seed=SEED)
                    run_model(model_path,l1,l2,loss)
print('all done')








# l1_list = [3,4,5,6]
# l2_list = [2,3,4,5]
# loss_list = ['mse','mae']
# df = gl.coregister_data('4')
# df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)

# df = df.sample(frac = 1,random_state = 0)
# df = df.reset_index().drop('index', axis = 1)
# df = df[list(df)[:-7]]

# trfeat = {}
# trlabs = {}
# tefeat = {}
# telabs = {}

# for n in df.index:
#     mask = df.index.isin([n])
#     trfeat[n] = df.loc[~mask].drop(['RGIId','Thickness'], axis = 1)
#     tefeat[n] = df.loc[mask].drop(['RGIId','Thickness'], axis = 1)
#     trlabs[n] = df['Thickness'].loc[~mask]
#     telabs[n] = df['Thickness'].loc[mask]

# results = pd.DataFrame(
#     columns = ['Coregistration','Arch','Metric','Loss']
# )

# for loss in loss_list:
    
#     res_dir = os.path.join(arch_test_path,loss,'final_results.pkl')
#     if os.path.isdir(res_dir) == True:
#         continue
#     else:
#         for l2 in l2_list:
#             for l1 in l1_list:
#                 if l2 >= l1:
#                     continue
#                 else:                    
#                     model = {}
#                     model_results = {}
#                     model_history = {}
#                     normalizer = {}
#                     for n in tqdm(df.index):
#                         model_path = os.path.join(
#                         home_path,arch_test_path ,loss,str(l1)+'-'+str(l2), str(n)
#                         )
#                         if os.path.isdir(model_path) == True:
#                             continue
#                             #                             set_global_determinism(seed=SEED)

#                         elif os.path.isdir(model_path) == False:
#                             set_global_determinism(seed=SEED)
#                             model, model_results, model_history = run_model(
#                             model_path, l1, l2, loss, n
#                             )
#                             break
# #                             print(model_results[n])
# #                             print(model_history([n])


        CenLat      CenLon  Slope  Zmin  Zmed  Zmax     Area  Aspect   Lmax
218  77.285700   14.994900    5.3   135   407   917  312.535     229  35580
259  78.107100   17.697800    9.5   110   445   773   39.032      87  13945
225  44.572747   80.173226   16.6  3447  3719  3997    1.361     203   2396
111  47.051000   12.409900   20.9  3053  3099  3150    0.052     122    280
178  80.580500   50.433100   13.6     0   263   421  512.015     143  15512
..         ...         ...    ...   ...   ...   ...      ...     ...    ...
170  78.767800   12.191900    9.1     2   439   974   57.860     325  16274
121  28.470000   85.816000    9.2  5710  5866  5944    1.327      30   1244
140  37.746680 -119.282740   23.9  3598  3694  3779    0.263     359    440
214  79.599900   12.207700   12.1    92   217   813   12.934      88   5762
240  50.049675   87.745522   19.3  2355  3301  3696    2.619      49   3827

[191 rows x 9 columns]        CenLat      CenLon  Slope  Zmin  Zmed  Zmax    Area  Aspe